In [24]:
# This is my capstone project for the summer analytics 2025
!pip install pathway bokeh --quiet
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
df= pd.read_csv('/content/dataset (1).csv')
# df=df.dropna() There are no columns with missing data
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [25]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [26]:
Vehicle_Weights = {
    "car": 2,
    "bike": 1,
    "truck": 3,
    "cycle":1
}

Traffic_Weights = {
    "low": 1,
    "medium": 2,
    "high": 3
}

# Create new columns using map
df["Vehicle_Weight"] = df["VehicleType"].map(Vehicle_Weights)
df["Traffic_Weight"] = df["TrafficConditionNearby"].map(Traffic_Weights).fillna(2).astype(int)
df

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,Vehicle_Weight,Traffic_Weight
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,2,1
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1,1
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,2,1
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,2,1
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,2,1
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,2,1
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,1,1
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00,2,1


In [27]:
# MODEL 1- LINEAR

In [28]:
df[["Timestamp", "Occupancy", "Capacity",'QueueLength','TrafficConditionNearby','VehicleType','Latitude','Longitude','IsSpecialDay','Vehicle_Weight','Traffic_Weight'
]].to_csv("parking_stream.csv", index=False)


class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    Traffic_Weight: int
    Vehicle_Weight: int
    Latitude: float
    Longitude: float
    IsSpecialDay: int


data_2 = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data_2 stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data_2.with_columns(
    t = data_2.Timestamp.dt.strptime(fmt),
    day = data_2.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)
df



,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,Vehicle_Weight,Traffic_Weight
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,2,1
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1,1
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,2,1
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,2,1
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,2,1
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,2,1
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,1,1
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00,2,1


In [29]:

df = pd.read_csv("parking_stream.csv")
print(df.columns)


Index(['Timestamp', 'Occupancy', 'Capacity', 'QueueLength',
       'TrafficConditionNearby', 'VehicleType', 'Latitude', 'Longitude',
       'IsSpecialDay', 'Vehicle_Weight', 'Traffic_Weight'],
      dtype='object')


In [30]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .reduce(
        t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
        occ_max=pw.reducers.max(pw.this.Occupancy),      # Highest occupancy observed in the window
        occ_min=pw.reducers.min(pw.this.Occupancy),      # Lowest occupancy observed in the window
        cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
    )
    .with_columns(
        # Compute the price using a simple dynamic pricing formula:
        #
        # Pricing Formula:
        #     price = base_price + demand_fluctuation
        #     where:
        #         base_price = 10 (fixed minimum price)
        #         demand_fluctuation = (occ_max - occ_min) / cap
        #
        # Intuition:
        # - The greater the difference between peak and low occupancy in a day,
        #   the more volatile the demand is, indicating potential scarcity.
        # - Dividing by capacity normalizes the fluctuation (to stay in [0,1] range).
        # - This fluctuation is added to the base price of 10 to set the final price.
        # - Example: If occ_max = 90, occ_min = 30, cap = 100
        #            => price = 10 + (90 - 30)/100 = 10 + 0.6 = 10.6

        price= 10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap
    )
)


In [31]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price(Linear Variation)",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [32]:
%%capture --no-display
pw.run()

Output()

In [33]:
# MODEL 2 - DEMAND BASED

In [34]:
delta_window = (
     data_with_time.windowby(
         pw.this.t,  # Event time column to use for windowing (parsed datetime)
         instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
         window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
         behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
     )
     .reduce(
         t=pw.this._pw_window_end,                        # Assign the end timestamp of each window
         cap=pw.reducers.max(pw.this.Capacity),           # Maximum capacity observed (typically constant per spot)
         count = pw.reducers.count(),
         special_day = pw.reducers.max(pw.this.IsSpecialDay),
         occ_sum = pw.reducers.sum(pw.this.Occupancy),
         queue_sum = pw.reducers.sum(pw.this.QueueLength),
         traffic_sum = pw.reducers.sum(pw.this.Traffic_Weight),
         vehicle_sum = pw.reducers.sum(pw.this.Vehicle_Weight),

         demand_min = pw.reducers.min(
            .7 * (pw.this.Occupancy / pw.this.Capacity) +
            .5 * pw.this.QueueLength -
            .9 * pw.this.Traffic_Weight +
            .4 * pw.this.IsSpecialDay +
            .6 * pw.this.Vehicle_Weight
        ),
        demand_max = pw.reducers.max(
            .7 * (pw.this.Occupancy / pw.this.Capacity) +
            .5 * pw.this.QueueLength -
            .9 * pw.this.Traffic_Weight +
            .4 * pw.this.IsSpecialDay +
            .6 * pw.this.Vehicle_Weight)

)
.with_columns(
    occ_avg = pw.this.occ_sum / pw.this.count,
    queue_avg = pw.this.queue_sum / pw.this.count,
    traffic_avg = pw.this.traffic_sum / pw.this.count,
    vehicle_avg = pw.this.vehicle_sum / pw.this.count,


    price_model2 = 10 + 4.5* (
        (
            1.5 * (pw.this.occ_sum / pw.this.count / pw.this.cap) +
            1.5 * (pw.this.queue_sum / pw.this.count) -
            3.0 * (pw.this.traffic_sum / pw.this.count) +
            1.5 * pw.this.special_day +
            2.0 * (pw.this.vehicle_sum / pw.this.count)
            - pw.this.demand_min
        ) / (pw.this.demand_max - pw.this.demand_min + 1e-6)
    )
)
)

In [37]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price (Demand Based)",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price_model2 evolves over time
    fig.line("t", "price_model2", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price_model2", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_model2_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [38]:
%%capture --no-display
pw.run()

Output()